In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
from itertools import chain
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
import joblib

# Mis utilitarios
from utils import *


In [3]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Concatenate, ConvLSTM2D, Reshape, Conv2D, Lambda
from tensorflow.keras.initializers import Constant
from tensorflow.keras.utils import plot_model

# Set Path

In [4]:
el_path_main = Path(os.getcwd()).parent
el_path_main

WindowsPath('e:/backup Asus G15 27_10_2021/Colegio de Mates Bourbaki/DLA2501 - Deep Learning Avanzado/Flood forecasting')

In [5]:
# En caso sea en google Colab
# from google.colab import drive
# drive.mount('/content/drive')

# Load Tensors

In [ ]:
X_gts_train, Y_train, X_gts_miss_train, X_ctry_lvl_train = joblib.load(el_path_main / '2. Datos/Processed data/TRAIN_data.pkl')
# X_gts_oot, Y_oot, X_gts_miss_oot, X_ctry_lvl_oot = joblib.load(el_path_main / '2. Datos/Processed data/OOT_data.pkl')

# El orden de los inputs en X_gts_train es:
# ['cld', 'dtr', 'frs', 'pet', 'pre', 'tmn', 'tmp', 'tmx', 'vap', 'wet']

In [6]:
# Reducimos el tamaño del dataset para que pueda ser ejecutado en local
desde_index = 700
X_gts_train = X_gts_train[desde_index:,:]
Y_train = Y_train[desde_index:,:]
X_gts_miss_train = X_gts_miss_train[desde_index:,:]
X_ctry_lvl_train = X_ctry_lvl_train[desde_index:,:]

In [7]:
# X_gts_train.shape, Y_train.shape, X_gts_miss_oot.shape, X_ctry_lvl_oot.shape
X_gts_train.shape, Y_train.shape, X_gts_miss_train.shape, X_ctry_lvl_train.shape

((477, 10, 96, 36, 28), (477, 12, 36, 28), (477, 10, 96, 36, 28), (477, 180))

In [8]:
def format_Y(Y):
    """
    Formatea Y para que sea un array de numpy con la forma (n_samples, n_classes)
    """
    Y = np.array(Y)
    Y = to_categorical(Y, num_classes=2)
    return Y

In [9]:
Y_train_f = format_Y(Y_train)
# Y_oot_f = format_Y(Y_oot)

In [10]:
X_gts_train.shape, Y_train.shape, X_gts_miss_train.shape, X_ctry_lvl_train.shape

((477, 10, 96, 36, 28), (477, 12, 36, 28), (477, 10, 96, 36, 28), (477, 180))

In [11]:
def gen_X_gts_for_model(X_gts, X_gts_miss):
    n_vars = X_gts.shape[1]
    lista_array = []
    for var in range(n_vars):
        X_tmp      = np.expand_dims(X_gts[:, var, :], -1)
        X_tmp_miss = np.expand_dims(X_gts_miss[:, var, :], -1)
        X = np.concat((X_tmp, X_tmp_miss), axis=-1)
        lista_array.append(X)
    return lista_array

In [12]:
# la_lista_de_arrays_oot = gen_X_gts_for_model(X_gts_oot, X_gts_miss_oot)
la_lista_de_arrays_train = gen_X_gts_for_model(X_gts_train, X_gts_miss_train)

In [13]:
la_lista_de_arrays_train[0].shape

(477, 96, 36, 28, 2)

# Version Simple

In [ ]:
n_vars_from_gts = X_gts_train.shape[1] # N of variables from GTS
n_times_for_lstm = X_gts_train.shape[2] # N of time steps usted for LSTM
n_lats = X_gts_train.shape[3] # N of latitudes
n_lons = X_gts_train.shape[4] # N of longitudes
vars_ctry_lvl = X_ctry_lvl_train.shape[1] # N of variables from country level data
n_codmes_fcast = Y_train.shape[1] # N of codmes for forecast

# Set of parameters that will be tunned
recurrent_dropout = 0.2
dropout = 0.2   
n_filters = 2
kernel_size = 3
include_miss_layer = True
dims_nn_ctry_lvl_tmp = [16] # La longitud de esta lista indica la cantidad de capas -1 (colcoar lista vacía para que sea de tamaño 1), los valores dentro indican el tamaño de la capa

# Set of parameters that are based on the previous ones
n_channels_conv_lstm = 2 if include_miss_layer else 1
dims_nn_ctry_lvl = dims_nn_ctry_lvl_tmp+[n_lats*n_lons*n_channels_conv_lstm] # La longitud de esta lista indica la cantidad de capas, los valores dentro indican el tamaño de la capa

In [15]:
def repeat_latent(x):
    return tf.repeat(x, repeats=n_codmes_fcast, axis=1)  # Repeat along time axis

In [16]:
mi_inp_gts = Input(shape=(n_times_for_lstm, n_lats, n_lons, n_channels_conv_lstm, ), name=f'input_gts')
mi_inp_ctry = Input(shape=(vars_ctry_lvl, ), name=f'input_ctry')

mi_lstm = ConvLSTM2D(filters=n_filters,
                     kernel_size=(kernel_size, kernel_size),
                     padding='same',
                     data_format = 'channels_last',
                     return_sequences=False,
                     recurrent_dropout=recurrent_dropout,
                     dropout=dropout,
                     name='conv_lstm_1')(mi_inp_gts)

mi_dense_ctry = Dense(36*28, activation='relu')(mi_inp_ctry)
mi_dense_ctry = Reshape((36, 28, 1))(mi_dense_ctry)

mi_encoded = Concatenate(axis=-1)([mi_lstm, mi_dense_ctry])
mi_encoded = Reshape((1, n_lats, n_lons, 3))(mi_encoded)
mi_encoded = Lambda(repeat_latent)(mi_encoded)

mi_decoder = ConvLSTM2D(filters=n_filters,
                        kernel_size=(kernel_size, kernel_size),
                        padding='same',
                        data_format = 'channels_last',
                        return_sequences=True,
                        recurrent_dropout=recurrent_dropout,
                        dropout=dropout,
                        name='conv_lstm_2')(mi_encoded)
output = TimeDistributed(Conv2D(2, (1, 1), activation='softmax'))(mi_decoder)
# Model
mi_simple_model = Model(inputs=[mi_inp_gts, mi_inp_ctry], outputs=output)
mi_simple_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
mi_simple_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ctry          │ (None, 180)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts           │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1008)      │    182,448 │ input_ctry[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm_1         │ (None, 36, 28, 2) │        296 │ input_gts[0][0]   │
│ (ConvLSTM2D)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 36, 28, 1) │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 36, 28, 3) │          0 │ conv_lstm_1[0][0… │
│ (Concatenate)       │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 1, 36, 28, │          0 │ concatenate[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 12, 36,    │          0 │ reshape_1[0][0]   │
│                     │ 28, 3)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm_2         │ (None, 12, 36,    │        368 │ lambda[0][0]      │
│ (ConvLSTM2D)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 12, 36,    │          6 │ conv_lstm_2[0][0] │
│ (TimeDistributed)   │ 28, 2)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 183,118 (715.30 KB)

 Trainable params: 183,118 (715.30 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
mi_simple_model.compile(optimizer='adam',         # el optimizador sirve para encontrar los pesos que minimizan la función de pérdida
                                            # adam: stochastic gradient descent adaptativo
                                            # https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
                  loss="categorical_crossentropy", # función que evalua que tan bien el algoritmo modela el conjunto de datos
                                                   # https://www.tensorflow.org/api_docs/python/tf/keras/losses
                  metrics=['accuracy'])

mi_simple_model_history = mi_simple_model.fit(x = [la_lista_de_arrays_train[0], X_ctry_lvl_train],
                         y = Y_train_f,
                        validation_split=0.2,
                        batch_size=128,
                        epochs=20,   # número de iteraciones sobre los datos
                        verbose=1)

Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.7760 - loss: 0.6810 - val_accuracy: 0.8446 - val_loss: 0.6561
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 996ms/step - accuracy: 0.7927 - loss: 0.6674 - val_accuracy: 0.8467 - val_loss: 0.6452
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 955ms/step - accuracy: 0.8014 - loss: 0.6565 - val_accuracy: 0.8613 - val_loss: 0.6347
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 931ms/step - accuracy: 0.8130 - loss: 0.6469 - val_accuracy: 0.8645 - val_loss: 0.6260
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 943ms/step - accuracy: 0.8197 - loss: 0.6381 - val_accuracy: 0.8744 - val_loss: 0.6180
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 956ms/step - accuracy: 0.8292 - loss: 0.6295 - val_accuracy: 0.8797 - val_loss: 0.6109
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 942ms/step - accuracy: 0.8354 - loss: 0.6213 - val_accuracy: 0.8875 - val_loss: 0.6028
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 964ms/step - accuracy: 0.8399 - loss: 0.6138 - val_accuracy: 0.9065 - val_loss: 0

In [19]:
mi_simple_model.save_weights(el_path_main/'4. Analisis/mi_simple_model.weights.h5')
# Epch20: 2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 488ms/step - accuracy: 0.7543 - loss: 0.6607 - val_accuracy: 0.7655 - val_loss: 0.6424


# Conv-LSTM

In [14]:
n_vars_from_gts = X_gts_train.shape[1] # N of variables from GTS
n_times_for_lstm = X_gts_train.shape[2] # N of time steps usted for LSTM
n_lats = X_gts_train.shape[3] # N of latitudes
n_lons = X_gts_train.shape[4] # N of longitudes
vars_ctry_lvl = X_ctry_lvl_train.shape[1] # N of variables from country level data
n_codmes_fcast = Y_train.shape[1] # N of codmes for forecast

# Set of parameters that will be tunned
recurrent_dropout = 0.2
dropout = 0.2   
n_filters = 2
kernel_size = 3
include_miss_layer = True
dims_nn_ctry_lvl_tmp = [16] # La longitud de esta lista indica la cantidad de capas -1 (colcoar lista vacía para que sea de tamaño 1), los valores dentro indican el tamaño de la capa

# Set of parameters that are based on the previous ones
n_channels_conv_lstm = 2 if include_miss_layer else 1
dims_nn_ctry_lvl = dims_nn_ctry_lvl_tmp+[n_lats*n_lons*n_channels_conv_lstm] # La longitud de esta lista indica la cantidad de capas, los valores dentro indican el tamaño de la capa

In [15]:
def repeat_latent(x):
    return tf.repeat(x, repeats=n_codmes_fcast, axis=1)  # Repeat along time axis

In [16]:
# Inputs for each variable from GTS    
map_inputs = [Input(shape=(n_times_for_lstm, n_lats, n_lons, n_channels_conv_lstm, ), name=f'input_gts_{i}') for i in range(n_vars_from_gts)]

# ConvLSTM encoding for maps
conv_outputs = [
    ConvLSTM2D(
            filters = n_filters,
            kernel_size = (kernel_size, kernel_size), # Tamaño del kernel
            padding = 'same', # Para que la salida tenga el mismo tamaño que la entrada
            data_format = 'channels_last', # Forma en que recibe los datos:Docu -> 5D tensor with shape: (samples, time, rows, cols, channels)
            return_sequences = False, # No retorna cada secuencia de salida
            # return_state=True, # Retorna los estados ocultos de la capa LSTM
            recurrent_dropout = recurrent_dropout, dropout = dropout  # Para evitar sobreajuste
            )(inp)
    for inp in map_inputs
]

# Input country level data
input_ctry_lvl = Input(shape=(vars_ctry_lvl,), name='input_ctry_lvl')
mi_dense_ctry_lvl = input_ctry_lvl

# Dense layers for country level data
for i in dims_nn_ctry_lvl:
    mi_dense_ctry_lvl = Dense(i, activation='relu')(mi_dense_ctry_lvl)

# Reshape so that it is compatible with the outputs of the LSTM.
mi_dense_ctry_lvl = Reshape((n_lats, n_lons, n_channels_conv_lstm))(mi_dense_ctry_lvl)

# Concatenate all the outputs of the LSTM layers and the dense layer for country level data
mi_concat = Concatenate()(conv_outputs+[mi_dense_ctry_lvl])
latent = Conv2D(n_filters, kernel_size=(kernel_size, kernel_size), padding='same', activation='relu')(mi_concat)
latent = Reshape((1, n_lats, n_lons, n_filters))(latent)


# Decoder
decoder_input = Lambda(repeat_latent)(latent) 
decoder = ConvLSTM2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), padding='same', return_sequences=True)(decoder_input)

# Multi-class output
output = TimeDistributed(Conv2D(2, (1, 1), activation='softmax'))(decoder)

# Model
model = Model(inputs = map_inputs + [input_ctry_lvl], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ctry_lvl      │ (None, 180)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16)        │      2,896 │ input_ctry_lvl[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_0         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_1         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_2         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_3         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_4         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_5         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_6         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_7         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_8         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_9         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 2016)      │     34,272 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d         │ (None, 36, 28, 2) │        296 │ input_gts_0[0][0] │
│ (ConvLSTM2D)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_1       │ (None, 36, 28, 2) │        296 │ input_gts_1[0][0] │
│ (ConvLSTM2D)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_2       │ (None, 36, 28, 2) │        296 │ input_gts_2[0][0] │
│ (ConvLSTM2D)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_3       │ (None, 36, 28, 2) │        296 │ input_gts_3[0][0] │
│ (ConvLSTM2D)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 40,828 (159.48 KB)

 Trainable params: 40,828 (159.48 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(optimizer='adam',         # el optimizador sirve para encontrar los pesos que minimizan la función de pérdida
                                            # adam: stochastic gradient descent adaptativo
                                            # https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
                  loss="categorical_crossentropy", # función que evalua que tan bien el algoritmo modela el conjunto de datos
                                                   # https://www.tensorflow.org/api_docs/python/tf/keras/losses
                  metrics=['accuracy'])

In [ ]:
history = model.fit(x = la_lista_de_arrays_train + [X_ctry_lvl_train],
                         y = Y_train_f,
                        validation_split=0.2,
                        batch_size=128,
                        epochs=20,   # número de iteraciones sobre los datos
                        verbose=1)

Epoch 1/20


In [ ]:
model.save_weights(el_path_main/'4. Analisis/model.weights.h5')

In [ ]:
# Si es en google Colab


# Benchmark (Logit)

In [5]:
df_gts_reescal = pd.read_csv(el_path_main / '2. Datos/Main gridded data/consol_df_peru_reescal.csv')
df_gts_reescal.sample(5)

,lon,lat,cld,dtr,frs,pet,pre,tmn,tmp,tmx,vap,wet,codmes,cuadrante_v1,cuadrante_v2,cuadrante_v3,mi_y,el_th_usado
1364376,-71.75,-8.25,0.783957,0.221277,NaN,0.415094,0.061161,0.900990,0.881459,0.801370,0.911585,0.433333,201310,1.0,1,1,0,NaN
1180273,-73.25,-1.75,0.745455,0.297872,NaN,0.396226,0.083383,0.873762,0.875380,0.825342,0.859756,0.600000,199807,1.0,1,1,0,NaN
306209,-81.25,-3.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192604,0.0,0,0,0,NaN
78367,-70.25,-4.75,0.773262,0.221277,NaN,0.301887,0.069920,0.863861,0.835866,0.750000,0.820122,0.500000,190706,1.0,1,1,0,NaN
1404522,-74.75,-11.25,0.840642,0.297872,NaN,0.452830,0.105305,0.754950,0.729483,0.660959,0.554878,0.533333,201702,1.0,1,1,0,NaN


In [6]:
df_ctry_lvl_reescal = pd.read_csv(el_path_main / '2. Datos/Main gridded data/df_ctry_lvl_reescal.csv')
df_ctry_lvl_reescal.sample(5)

,codmes,country,cld,dtr,frs,pet,pre,tmn,tmp,tmx,vap,wet,codmes_d,pre_lag,wet_lag
6463,193209,Canada,0.646204,0.300000,0.432258,0.218750,0.081401,0.596886,0.583904,0.563545,0.222951,0.354515,1932-09-01,61.6,12.1
4700,192401,Guiana,0.775444,0.171429,0.000000,0.421875,0.436606,0.951557,0.919521,0.876254,0.842623,0.652174,1924-01-01,364.9,23.1
16273,198010,Chile,0.306947,0.364286,0.296774,0.453125,0.063641,0.641869,0.636986,0.622074,0.236066,0.086957,1980-10-01,77.1,3.9
8316,194110,Canada,0.691438,0.200000,0.800000,0.093750,0.081237,0.463668,0.441781,0.413043,0.124590,0.381271,1941-10-01,48.1,11.4
15458,197610,Colombia,0.940226,0.278571,0.000000,0.406250,0.472126,0.925606,0.905822,0.874582,0.865574,0.635452,1976-10-01,332.0,19.7
